# OFA³: Automatic Selection of the Best Non-dominated Sub-networks for Ensembles

- **Description**: 
  - This Jupyter notebook runs the OFA³ selection method for obtaining efficient ensembles. 
  - The search is guided by an evolutionary multi-objective optimization algorithm (EMOA). Here we compare three EMOA:
    - NSGA-II
    - SMS-EMOA
    - SPEA2
  - The objective functions to be optimized are the ImageNet top-1 % accuracy and latency.
  - In these experiments we consider the latency of the ensemble equals to the **<ins>summed latency</ins>** of all neural networks in the ensemble.

- **Input**: The input of the OFA³ selection method is a set of neural networks.

- **Output**: The output of the OFA³ selection method is a set of efficient ensembles. The number of the components of the ensemble and the components themselves (i.e., the neural networks that compose the ensemble) are automatically found by the algorithm. 

- **arXiv link**: TBA

- **Author**: TBA (hidden due to blind review)
- **email**: TBA (hidden due to blind review)

# Imports

In [1]:
# general
import os
import time
import math
import random
import pickle
from tqdm.notebook import tqdm

# AI/ML/NN
import torch
import numpy as np
import pandas as pd
from torchvision import datasets, transforms

# pymoo
from pymoo.core.individual import Individual
from pymoo.core.population import Population
from pymoo.core.variable import Choice
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.sampling import Sampling
from pymoo.core.callback import Callback
from pymoo.operators.mutation.rm import ChoiceRandomMutation
from pymoo.operators.crossover.ux import UniformCrossover
from pymoo.termination import get_termination
from pymoo.util.display.output import Output
from pymoo.util.display.column import Column
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.sms import SMSEMOA
from pymoo.algorithms.moo.spea2 import SPEA2
from pymoo.optimize import minimize
#from pymoo.indicators.hv import HV

In [2]:
# set random seeds for reproducibility
random_seed = 1
random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)
#----------------------------
seed1 = 1
seed2 = 2
seed3 = 3

In [3]:
# set device to use GPU or CPU
cuda_available = torch.cuda.is_available()
if cuda_available:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    torch.cuda.manual_seed(random_seed)
    cuda0 = torch.device('cuda:0')
    print("Using GPU.")
else:
    print("Using CPU.")

Using GPU.


In [4]:
# colors used in matplotlib curves
blue   = '#1f77b4'
orange = '#ff7f0e'
green  = '#2ca02c'
red    = '#d62728'
purple = '#9467bd'
brown  = '#8c564b'
pink   = '#e377c2'
gray   = '#7f7f7f'
olive  = '#bcbd22'
cyan   = '#17becf'
black  = '#000000'

# Get OFA network

In [5]:
#ofa_network = ofa_net("ofa_mbv3_d234_e346_k357_w1.2", pretrained=True)
# ofa_network2 = torch.load(model_dir='~/model/ofa_mbv3_d234_e346_k357_w1.2')

# Dataset & DataLoader

In [6]:
# ImageNet Full
imagenet_data_path = "~/dataset/imagenet/"
#----------------------------
# ImageNet subset
#imagenet_data_path = "~/dataset/imagenet_1k"

In [7]:
# os.makedirs(imagenet_data_path, exist_ok=True)
# download_url('https://hanlab.mit.edu/files/OnceForAll/ofa_cvpr_tutorial/imagenet_1k.zip', model_dir='data')
#! cd data && unzip imagenet_1k 1>/dev/null && cd ..
#! cp -r data/imagenet_1k/* $imagenet_data_path
#! rm -rf data

In [8]:
# The following function build the data transforms for test
def build_val_transform(size):
    return transforms.Compose(
        [
            transforms.Resize(int(math.ceil(size / 0.875))),
            transforms.CenterCrop(size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )

In [9]:
#imagenet_train = datasets.ImageFolder(root=os.path.join(imagenet_data_path, "train"), transform=build_val_transform(224))
imagenet_train_subset = datasets.ImageFolder(root=os.path.join(imagenet_data_path, "train_subset_50k"), transform=build_val_transform(224))
imagenet_val = datasets.ImageFolder(root=os.path.join(imagenet_data_path, "val"), transform=build_val_transform(224))

In [10]:
## this dataloader is for the training set --> used to guide the search
#data_loader_train = torch.utils.data.DataLoader(
#    imagenet_train,
#    batch_size=256,   # evaluation batch size
#    shuffle=False,    # evaluation only
#    num_workers=4,    # number of workers for the data loader
#    pin_memory=True,
#    drop_last=False,
#)
#print("The ImageNet dataloader for the training set is ready.")

In [11]:
# this dataloader is for the 50k subset of the training set --> used to generate probability table
data_loader_train_subset = torch.utils.data.DataLoader(
    datasets.ImageFolder(root=os.path.join(imagenet_data_path, "train_subset_50k"), transform=build_val_transform(224)),
    batch_size=1_024, # evaluation batch size
    shuffle=False,    # evaluation only
    num_workers=4,    # number of workers for the data loader
    pin_memory=True,
    drop_last=False,
)
print("The ImageNet dataloader for the training set is ready.")

The ImageNet dataloader for the training set is ready.


In [12]:
# this dataloader is for the validation set --> used to measure performance
#data_loader_val = torch.utils.data.DataLoader(
#    imagenet_val,
#    batch_size=1_024, # test batch size
#    shuffle=False,    # evaluation only
#    num_workers=4,    # number of workers for the data loader
#    pin_memory=True,
#    drop_last=False,
#)
#print("The ImageNet dataloader for the validation set is ready.")

# Load the probability & class outputs of the OFA² models

In [13]:
# load last population of OFA² search
with open('ofa2_nsga2.pickle', 'rb') as f:
    ofa2_nsga2 = pickle.load(f)
#----------------------------
# get latency of the models
ofa2_nsga2_latency = ofa2_nsga2.get('F')[:,0]

These tables are obtained evaluating the architectures found by the OFA² search

In [14]:
n_outputs = 5                                   # number of outputs considered to build the cumulative table
n_models = len(ofa2_nsga2)                      # OFA² last population size: 100
ds_size = len(imagenet_train_subset)            # ImageNet training set size: 50,000
n_classes = len(imagenet_train_subset.classes)  # ImageNet classes: 1,000 
#----------------------------
#ds_size = len(imagenet_train_subset)            # ImageNet training set size: 1,281,167
#n_classes = len(imagenet_train_subset.classes)  # ImageNet classes: 1,000 

In [15]:
# create class and probabilities tables
top5_class = np.empty((n_models, ds_size, n_outputs), dtype=np.int16)   # shape = (100 x 1281167 x 5)
top5_prob = np.empty((n_models, ds_size, n_outputs), dtype=np.float32)  # shape = (100 x 1281167 x 5)
#-------------------------------------------------
# fill table with data from csv files of the 100 models found by OFA² search
for i in tqdm(range(n_models)):
    top5_class[i] = pd.read_csv('ofa2_models_output_subset_50k/OFA2_model_' + str(i).zfill(3) + '_class.csv', usecols=[0,1,2,3,4], header=None).values
    top5_prob[i] = pd.read_csv('ofa2_models_output_subset_50k/OFA2_model_' + str(i).zfill(3) + '_prob.csv', usecols=[0,1,2,3,4], header=None).values
#-------------------------------------------------
# ground truth labels
labels = pd.read_csv('ofa2_models_output_subset_50k/OFA2_model_' + str(0).zfill(3) + '_class.csv', usecols=[5], header=None).values.squeeze()

  0%|          | 0/100 [00:00<?, ?it/s]

# Fill the cumulative probability lookup table

In [16]:
# create cumulative lookup table of probabilities: shape = (100 x 1281167 x 1000)
cumulative_lut = np.empty((n_models, ds_size, n_classes), dtype=np.float32)

In [17]:
# start measuring time
start_time = time.time()
#-------------------------------------------------
# loop over models (0~99)
for model_id in tqdm(range(n_models), position=1):
    # loop over each image output class (0~1281166): get lines of top5_class table
    #for i, output_classes in enumerate(tqdm(top5_class[model_id], position=0, leave=False)):
    for i, output_classes in enumerate(top5_class[model_id]):
        # loop over each top-5 outputs (0~4): get rows of top5_class table & respective rows of top5_prob
        for j, topk in enumerate(output_classes):
            cumulative_lut[model_id, i, topk] += top5_prob[model_id, i, j]
#-------------------------------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The creation of the cumulative probability lookup tables took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

  0%|          | 0/100 [00:00<?, ?it/s]

The creation of the cumulative probability lookup tables took 00h00m23s to finish.


# Hyperparameters

In [18]:
""" Hyperparameters
- P: size of the population in each generation (number of individuals)
- N: number of generations to run the algorithm
- mutate_prob: probability of gene mutation in the evolutionary search
"""
P = 100
#N = 1_000
N = 200
mutation_prob = 0.01

# Sum Latency (components: n > 2)

## Individual & Population

In [19]:
class EnsembleIndividual(Individual):
    def __init__(self, ensemble_selection, config=None, **kwargs):
        super().__init__(config=None, **kwargs)
        self.X = ensemble_selection

In [20]:
# individuals with all genes equals to "1"
pop = Population(individuals=[EnsembleIndividual(ind) for ind in np.ones((P,P))])

## Search Space & Problem

In [21]:
# participation
part_var = Choice(options=[0, 1])
#----------------------------
# search space
ensemble_size = P

In [22]:
class EnsembleProblemSoft(ElementwiseProblem):
    def __init__(self, ensemble_size, cumulative_table, latency_table, label, device, metric='max'):
        super().__init__(
            # n_var = 100,
            vars=ensemble_size * [part_var],
            n_obj=2,
            n_constr=1,
        )
        self.device = device  # allocate in GPU 
        self.label = torch.tensor(label).to(self.device)
        self.val_size = torch.tensor(len(label)).to(self.device)
        self.N_CLASSES = 1_000
        self.latency_table = latency_table
        self.metric = metric
        # reshape to (50k x 1k x 100)
        self.cumulative_table = torch.permute(torch.tensor(cumulative_table), (1, 2, 0)).to(self.device)
        
        
    def calc_ensemble_lat_sum(self, x):
        return np.inner(x, self.latency_table)
    
    
    def calc_ensemble_lat_max(self, x):
        return max(np.multiply(x, self.latency_table))
    
    
    def calc_ensemble_accuracy(self, x):
        ind = torch.tensor(x, device=self.device, dtype=torch.float32)
        out = torch.matmul(self.cumulative_table, ind)
        acc = torch.count_nonzero(torch.argmax(out, axis=1) == self.label, axis=0) / self.val_size * 100
        return acc.item()
    
    
    def _evaluate(self, x, out, *args, **kwargs):
        if self.metric == 'sum':
            f1 = self.calc_ensemble_lat_sum(x)   # sum latency
        else:
            f1 = self.calc_ensemble_lat_max(x)   # max latency
        acc = self.calc_ensemble_accuracy(x)
        f2 = 100 - acc
        g1 = 2 - np.count_nonzero(x)
        out["F"] = np.column_stack([f1, f2])
        out["G"] = [g1]

## Operators

In [23]:
# Mutation: Random
ensemble_mutation_rc = ChoiceRandomMutation(prob=1.0, prob_var=mutation_prob)

In [24]:
# Crossover (Recombination): Uniform
ensemble_crossover_ux = UniformCrossover(prob=1.0)
parents = [[pop[0], pop[1]] for _ in range(1)]

In [25]:
# Sampling: Random
class EnsembleSampling(Sampling):
    def _do(self, ensemble_problem, n_samples, **kwargs):
        return [
            [np.random.choice(var.options) for var in ensemble_problem.vars]
            for _ in range(n_samples)
        ]
#----------------------------
ensemble_sampling = EnsembleSampling()

## Termination

In [26]:
# convergence
#termination_default = DefaultMultiObjectiveTermination(xtol=1e-8, cvtol=1e-6, ftol=0.0025, period=30, n_max_gen=1000, n_max_evals=100000)
#----------------------------
# generations
termination_gen = get_termination("n_gen", N)

## Algorithm & Optimization

In [27]:
class MyOutput(Output):

    def __init__(self):
        super().__init__()
        self.n_gen = Column("n_gen", width=5)
        #self.g_mean = Column("g_mean", width=6)
        #self.g_min = Column("g_mean", width=6)
        #self.g_max = Column("g_mean", width=6)
        self.lat_min = Column("lat_min", width=15)
        self.lat_max = Column("lat_max", width=15)
        self.acc_min = Column("acc_min", width=15)
        self.acc_max = Column("acc_max", width=15)
        self.acc_mean = Column("acc_mean", width=15)
        #self.columns = [self.n_gen, self.g_mean, self.lat_min, self.lat_max, self.acc_min, self.acc_max, self.acc_mean]
        #self.columns = [self.n_gen, self.g_min, self.g_max, self.lat_min, self.lat_max, self.acc_min, self.acc_max, self.acc_mean]
        self.columns = [self.n_gen, self.lat_min, self.lat_max, self.acc_min, self.acc_max, self.acc_mean]
        #self.columns = [self.n_gen, self.g_mean]#, self.lat_min, self.lat_max, self.acc_min, self.acc_max, self.acc_mean]
        #self.columns = [self.n_gen, self.lat_min, self.lat_max, self.acc_min, self.acc_max, self.acc_mean]

    def update(self, algorithm):
        super().update(algorithm)
        self.n_gen.set(algorithm.n_gen)
        #self.g_mean.set(2 - algorithm.pop.get("G"))
        #self.g_mean.set(np.mean(2 - algorithm.pop.get("G")[:]).item())
        #self.g_min.set(np.abs(np.min(algorithm.pop.get("G"))))
        #self.g_max.set(np.abs(np.max(algorithm.pop.get("G"))))
        self.lat_min.set(np.min(algorithm.pop.get("F")[:,0]))
        self.lat_max.set(np.max(algorithm.pop.get("F")[:,0]))
        self.acc_min.set(np.min(100 - algorithm.pop.get("F")[:,1]))
        self.acc_max.set(np.max(100 - algorithm.pop.get("F")[:,1]))
        self.acc_mean.set(np.mean(100 - algorithm.pop.get("F")[:,1]))

### Summed Latency

In [28]:
ensemble_problem_sum = EnsembleProblemSoft(P, cumulative_lut, ofa2_nsga2_latency, labels, device=cuda0, metric='sum')

#### NSGA-II

In [29]:
class Nsga2Callback(Callback):
    
    def __init__(self, metric='max', seed=1) -> None:
        super().__init__()
        self.data = []
        self.metric = metric
        self.seed = seed
        
    def notify(self, algorithm):
        pop_data = algorithm.n_gen, algorithm.pop.get("X"), algorithm.pop.get("F"), algorithm.pop.get("G")
        filename = 'gen_' + str(algorithm.n_gen).zfill(4) + '.pickle'
        with open(os.path.join('ofa3_ensemble', 'NSGA-II', 'soft-lat' + self.metric + '-generation', 'seed_' + str(self.seed), filename), 'bw') as f:
            pickle.dump(pop_data, f)

In [30]:
ensemble_algorithm_nsga2 = NSGA2(
    pop_size=P,
    sampling=Population(individuals=[EnsembleIndividual(ind) for ind in np.ones((P,P))]),
    #sampling=Population(individuals=[EnsembleIndividual(ind) for ind in np.identity(P)]),
    crossover=ensemble_crossover_ux,
    mutation=ensemble_mutation_rc,
)

##### seed 1

In [31]:
torch.manual_seed(seed1)
np.random.seed(seed1)
random.seed(seed1)

In [32]:
# start measuring time
start_time = time.time()
#----------------------------
ensemble_nsga2_sum_seed1 = minimize(
    ensemble_problem_sum,
    ensemble_algorithm_nsga2,
    callback=Nsga2Callback(metric='sum', seed=seed1),
    termination=termination_gen,
    seed=seed1,
    verbose=True,
    #verbose=False,
    output=MyOutput(),
    save_history=False,
)
#----------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The optimization took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

n_gen |     lat_min     |     lat_max     |     acc_min     |     acc_max     |     acc_mean   
    1 |  2.37374098E+03 |  2.37374098E+03 |  9.10159988E+01 |  9.10159988E+01 |  9.10159988E+01
    2 |  2.27064722E+03 |  2.36385063E+03 |  9.09860001E+01 |  9.10440063E+01 |  9.10174000E+01
    3 |  2.20832576E+03 |  2.36227662E+03 |  9.09720001E+01 |  9.10559998E+01 |  9.10201397E+01
    4 |  2.16591905E+03 |  2.34950455E+03 |  9.09720001E+01 |  9.10559998E+01 |  9.10208997E+01
    5 |  2.14632910E+03 |  2.34950455E+03 |  9.09720001E+01 |  9.10680008E+01 |  9.10231599E+01
    6 |  2.11563686E+03 |  2.31798085E+03 |  9.09619980E+01 |  9.10780029E+01 |  9.10266206E+01
    7 |  2.09141480E+03 |  2.30484587E+03 |  9.09580002E+01 |  9.10780029E+01 |  9.10301401E+01
    8 |  1.98965593E+03 |  2.28829294E+03 |  9.09659958E+01 |  9.10780029E+01 |  9.10351401E+01
    9 |  1.97408810E+03 |  2.28829294E+03 |  9.09659958E+01 |  9.10839996E+01 |  9.10443400E+01
   10 |  1.91582030E+03 |  2.23811297E+0

In [33]:
del ensemble_nsga2_sum_seed1
torch.cuda.empty_cache()

##### seed 2

In [34]:
torch.manual_seed(seed2)
np.random.seed(seed2)
random.seed(seed2)

In [35]:
# start measuring time
start_time = time.time()
#----------------------------
ensemble_nsga2_sum_seed2 = minimize(
    ensemble_problem_sum,
    ensemble_algorithm_nsga2,
    callback=Nsga2Callback(metric='sum', seed=seed2),
    termination=termination_gen,
    seed=seed2,
    verbose=True,
    #verbose=False,
    output=MyOutput(),
    save_history=False,
)
#----------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The optimization took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

n_gen |     lat_min     |     lat_max     |     acc_min     |     acc_max     |     acc_mean   
    1 |  2.37374098E+03 |  2.37374098E+03 |  9.10159988E+01 |  9.10159988E+01 |  9.10159988E+01
    2 |  2.29969812E+03 |  2.36373887E+03 |  9.09940033E+01 |  9.10380020E+01 |  9.10174398E+01
    3 |  2.21726753E+03 |  2.35203689E+03 |  9.09899979E+01 |  9.10559998E+01 |  9.10213996E+01
    4 |  2.17684294E+03 |  2.35203689E+03 |  9.09899979E+01 |  9.10640030E+01 |  9.10243405E+01
    5 |  2.12447397E+03 |  2.33277079E+03 |  9.09899979E+01 |  9.10640030E+01 |  9.10258801E+01
    6 |  2.11352541E+03 |  2.33277079E+03 |  9.09700012E+01 |  9.10699997E+01 |  9.10309199E+01
    7 |  2.05208570E+03 |  2.29304691E+03 |  9.09680023E+01 |  9.10759964E+01 |  9.10291201E+01
    8 |  2.03789415E+03 |  2.27973969E+03 |  9.09440002E+01 |  9.10859985E+01 |  9.10364400E+01
    9 |  1.95619232E+03 |  2.20843096E+03 |  9.09440002E+01 |  9.10979996E+01 |  9.10392204E+01
   10 |  1.86023286E+03 |  2.18524854E+0

In [36]:
del ensemble_nsga2_sum_seed2
torch.cuda.empty_cache()

##### seed 3

In [37]:
torch.manual_seed(seed3)
np.random.seed(seed3)
random.seed(seed3)

In [38]:
# start measuring time
start_time = time.time()
#----------------------------
ensemble_nsga2_sum_seed3 = minimize(
    ensemble_problem_sum,
    ensemble_algorithm_nsga2,
    callback=Nsga2Callback(metric='sum', seed=seed3),
    termination=termination_gen,
    seed=seed3,
    verbose=True,
    #verbose=False,
    output=MyOutput(),
    save_history=False,
)
#----------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The optimization took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

n_gen |     lat_min     |     lat_max     |     acc_min     |     acc_max     |     acc_mean   
    1 |  2.37374098E+03 |  2.37374098E+03 |  9.10159988E+01 |  9.10159988E+01 |  9.10159988E+01
    2 |  2.28794335E+03 |  2.36373887E+03 |  9.09899979E+01 |  9.10419998E+01 |  9.10162394E+01
    3 |  2.24999323E+03 |  2.36351911E+03 |  9.09840012E+01 |  9.10540009E+01 |  9.10176201E+01
    4 |  2.19928786E+03 |  2.35154041E+03 |  9.09739990E+01 |  9.10619965E+01 |  9.10240799E+01
    5 |  2.13832534E+03 |  2.34782542E+03 |  9.09820023E+01 |  9.10699997E+01 |  9.10277997E+01
    6 |  2.05088109E+03 |  2.33145326E+03 |  9.09519958E+01 |  9.10800018E+01 |  9.10359801E+01
    7 |  2.05088109E+03 |  2.32283787E+03 |  9.09519958E+01 |  9.10820007E+01 |  9.10420999E+01
    8 |  2.03037264E+03 |  2.27096307E+03 |  9.09519958E+01 |  9.10859985E+01 |  9.10420602E+01
    9 |  1.96117690E+03 |  2.27096307E+03 |  9.09519958E+01 |  9.10899963E+01 |  9.10462199E+01
   10 |  1.93050531E+03 |  2.27096307E+0

In [39]:
del ensemble_nsga2_sum_seed3
torch.cuda.empty_cache()

#### SMS-EMOA

In [40]:
class SmsemoaCallback(Callback):
    
    def __init__(self, metric='max', seed=1) -> None:
        super().__init__()
        self.data = []
        self.metric = metric
        self.seed = seed
        
    def notify(self, algorithm):
        pop_data = algorithm.n_gen, algorithm.pop.get("X"), algorithm.pop.get("F"), algorithm.pop.get("G")
        filename = 'gen_' + str(algorithm.n_gen).zfill(4) + '.pickle'
        with open(os.path.join('ofa3_ensemble', 'SMS-EMOA', 'soft-lat' + self.metric + '-generation', 'seed_' + str(self.seed), filename), 'bw') as f:
            pickle.dump(pop_data, f)

In [41]:
ensemble_algorithm_smsemoa = SMSEMOA(
    pop_size=P,
    # sampling=ensemble_sampling,
    # sampling=Population(individuals=[EnsembleIndividual(ind) for ind in np.identity(P)]),
    sampling=Population(individuals=[EnsembleIndividual(ind) for ind in np.ones((P,P))]),
    crossover=ensemble_crossover_ux,
    mutation=ensemble_mutation_rc,
)

##### seed 1

In [42]:
torch.manual_seed(seed1)
np.random.seed(seed1)
random.seed(seed1)

In [43]:
# start measuring time
start_time = time.time()
#----------------------------
ensemble_smsemoa_sum_seed1 = minimize(
    ensemble_problem_sum,
    ensemble_algorithm_smsemoa,
    callback=SmsemoaCallback(metric='sum', seed=seed1),
    termination=termination_gen,
    seed=seed1,
    verbose=True,
    #verbose=False,
    output=MyOutput(),
    save_history=False,
)
#----------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The optimization took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

n_gen |     lat_min     |     lat_max     |     acc_min     |     acc_max     |     acc_mean   
    1 |  2.37374098E+03 |  2.37374098E+03 |  9.10159988E+01 |  9.10159988E+01 |  9.10159988E+01
    2 |  2.27064722E+03 |  2.36385063E+03 |  9.09860001E+01 |  9.10440063E+01 |  9.10174000E+01
    3 |  2.24143816E+03 |  2.36334953E+03 |  9.09860001E+01 |  9.10440063E+01 |  9.10204198E+01
    4 |  2.19331447E+03 |  2.35105365E+03 |  9.09860001E+01 |  9.10660019E+01 |  9.10253201E+01
    5 |  2.11582965E+03 |  2.34950455E+03 |  9.09899979E+01 |  9.10660019E+01 |  9.10292005E+01
    6 |  2.11582965E+03 |  2.34120145E+03 |  9.09720001E+01 |  9.10699997E+01 |  9.10306601E+01
    7 |  2.07727577E+03 |  2.31437012E+03 |  9.09820023E+01 |  9.10699997E+01 |  9.10340404E+01
    8 |  2.05175609E+03 |  2.28548254E+03 |  9.09759979E+01 |  9.10820007E+01 |  9.10416604E+01
    9 |  1.98965678E+03 |  2.25952015E+03 |  9.09499969E+01 |  9.10920029E+01 |  9.10451801E+01
   10 |  1.94706128E+03 |  2.25952015E+0

In [44]:
del ensemble_smsemoa_sum_seed1
torch.cuda.empty_cache()

##### seed 2

In [45]:
torch.manual_seed(seed2)
np.random.seed(seed2)
random.seed(seed2)

In [46]:
# start measuring time
start_time = time.time()
#----------------------------
ensemble_smsemoa_sum_seed2 = minimize(
    ensemble_problem_sum,
    ensemble_algorithm_smsemoa,
    callback=SmsemoaCallback(metric='sum', seed=seed2),
    termination=termination_gen,
    seed=seed2,
    verbose=True,
    #verbose=False,
    output=MyOutput(),
    save_history=False,
)
#----------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The optimization took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

n_gen |     lat_min     |     lat_max     |     acc_min     |     acc_max     |     acc_mean   
    1 |  2.37374098E+03 |  2.37374098E+03 |  9.10159988E+01 |  9.10159988E+01 |  9.10159988E+01
    2 |  2.29969812E+03 |  2.36373887E+03 |  9.09940033E+01 |  9.10380020E+01 |  9.10174398E+01
    3 |  2.24701213E+03 |  2.36334953E+03 |  9.09899979E+01 |  9.10499954E+01 |  9.10211797E+01
    4 |  2.20250286E+03 |  2.34933748E+03 |  9.09899979E+01 |  9.10640030E+01 |  9.10271001E+01
    5 |  2.16612379E+03 |  2.34782542E+03 |  9.09820023E+01 |  9.10640030E+01 |  9.10312606E+01
    6 |  2.07491624E+03 |  2.32255288E+03 |  9.09640045E+01 |  9.10740051E+01 |  9.10353602E+01
    7 |  2.04676898E+03 |  2.32739670E+03 |  9.09640045E+01 |  9.10780029E+01 |  9.10396201E+01
    8 |  2.04244830E+03 |  2.32255288E+03 |  9.09640045E+01 |  9.10880051E+01 |  9.10412199E+01
    9 |  1.95679194E+03 |  2.27690381E+03 |  9.09739990E+01 |  9.10880051E+01 |  9.10460800E+01
   10 |  1.87570692E+03 |  2.25048626E+0

In [47]:
del ensemble_smsemoa_sum_seed2
torch.cuda.empty_cache()

##### seed 3

In [48]:
torch.manual_seed(seed3)
np.random.seed(seed3)
random.seed(seed3)

In [49]:
# start measuring time
start_time = time.time()
#----------------------------
ensemble_smsemoa_sum_seed3 = minimize(
    ensemble_problem_sum,
    ensemble_algorithm_smsemoa,
    callback=SmsemoaCallback(metric='sum', seed=seed3),
    termination=termination_gen,
    seed=seed3,
    verbose=True,
    #verbose=False,
    output=MyOutput(),
    save_history=False,
)
#----------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The optimization took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

n_gen |     lat_min     |     lat_max     |     acc_min     |     acc_max     |     acc_mean   
    1 |  2.37374098E+03 |  2.37374098E+03 |  9.10159988E+01 |  9.10159988E+01 |  9.10159988E+01
    2 |  2.28794335E+03 |  2.36373887E+03 |  9.09899979E+01 |  9.10419998E+01 |  9.10162394E+01
    3 |  2.21776994E+03 |  2.36351911E+03 |  9.09899979E+01 |  9.10459976E+01 |  9.10189800E+01
    4 |  2.16027547E+03 |  2.34992778E+03 |  9.09759979E+01 |  9.10559998E+01 |  9.10213799E+01
    5 |  2.16027547E+03 |  2.34992778E+03 |  9.09759979E+01 |  9.10579987E+01 |  9.10250600E+01
    6 |  2.04151847E+03 |  2.33644354E+03 |  9.09519958E+01 |  9.10680008E+01 |  9.10259397E+01
    7 |  2.04151847E+03 |  2.29264748E+03 |  9.09499969E+01 |  9.10740051E+01 |  9.10299395E+01
    8 |  1.95558905E+03 |  2.27570645E+03 |  9.09499969E+01 |  9.10920029E+01 |  9.10410603E+01
    9 |  1.91216487E+03 |  2.24534751E+03 |  9.09659958E+01 |  9.10960007E+01 |  9.10464201E+01
   10 |  1.91216487E+03 |  2.20872409E+0

In [50]:
del ensemble_smsemoa_sum_seed3
torch.cuda.empty_cache()

#### SPEA2

In [51]:
class Spea2Callback(Callback):
    
    def __init__(self, metric='max', seed=1) -> None:
        super().__init__()
        self.data = []
        self.metric = metric
        self.seed = seed
        
    def notify(self, algorithm):
        pop_data = algorithm.n_gen, algorithm.pop.get("X"), algorithm.pop.get("F"), algorithm.pop.get("G")
        filename = 'gen_' + str(algorithm.n_gen).zfill(4) + '.pickle'
        with open(os.path.join('ofa3_ensemble', 'SPEA2', 'soft-lat' + self.metric + '-generation', 'seed_' + str(self.seed), filename), 'bw') as f:
            pickle.dump(pop_data, f)

In [52]:
ensemble_algorithm_spea2 = SPEA2(
    pop_size=P,
    #sampling=ensemble_sampling,
    #sampling=Population(individuals=[EnsembleIndividual(ind) for ind in np.identity(P)]),
    sampling=Population(individuals=[EnsembleIndividual(ind) for ind in np.ones((P,P))]),
    crossover=ensemble_crossover_ux,
    mutation=ensemble_mutation_rc,
)

##### seed 1

In [53]:
torch.manual_seed(seed1)
np.random.seed(seed1)
random.seed(seed1)

In [54]:
# start measuring time
start_time = time.time()
#----------------------------
ensemble_spea2_sum_seed1 = minimize(
    ensemble_problem_sum,
    ensemble_algorithm_spea2,
    callback=Spea2Callback(metric='sum', seed=seed1),
    termination=termination_gen,
    seed=seed1,
    verbose=True,
    #verbose=False,
    output=MyOutput(),
    save_history=False,
)
#----------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The optimization took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

n_gen |     lat_min     |     lat_max     |     acc_min     |     acc_max     |     acc_mean   
    1 |  2.37374098E+03 |  2.37374098E+03 |  9.10159988E+01 |  9.10159988E+01 |  9.10159988E+01


/opt/conda/lib/python3.10/site-packages/pymoo/algorithms/moo/spea2.py:62: RuntimeWarning: invalid value encountered in divide
  _F = (F - ideal) / (nadir - ideal)


    2 |  2.27064722E+03 |  2.36385063E+03 |  9.09860001E+01 |  9.10440063E+01 |  9.10174000E+01
    3 |  2.20277951E+03 |  2.36334953E+03 |  9.09840012E+01 |  9.10699997E+01 |  9.10189398E+01
    4 |  2.16546215E+03 |  2.35326700E+03 |  9.09799957E+01 |  9.10699997E+01 |  9.10221002E+01
    5 |  2.12883241E+03 |  2.33468033E+03 |  9.09659958E+01 |  9.10699997E+01 |  9.10259396E+01
    6 |  2.01884188E+03 |  2.33468033E+03 |  9.09599991E+01 |  9.10920029E+01 |  9.10329996E+01
    7 |  2.01337006E+03 |  2.33468033E+03 |  9.09599991E+01 |  9.10920029E+01 |  9.10374205E+01
    8 |  1.96891952E+03 |  2.33468033E+03 |  9.09739990E+01 |  9.10979996E+01 |  9.10428809E+01
    9 |  1.92788825E+03 |  2.25937307E+03 |  9.09739990E+01 |  9.10979996E+01 |  9.10519405E+01
   10 |  1.90060184E+03 |  2.23752363E+03 |  9.09780045E+01 |  9.11040039E+01 |  9.10590204E+01
   11 |  1.82503468E+03 |  2.20805911E+03 |  9.09739990E+01 |  9.11080017E+01 |  9.10681804E+01
   12 |  1.70691870E+03 |  2.16547602E+0

In [55]:
del ensemble_spea2_sum_seed1
torch.cuda.empty_cache()

##### seed 2

In [56]:
torch.manual_seed(seed2)
np.random.seed(seed2)
random.seed(seed2)

In [57]:
# start measuring time
start_time = time.time()
#----------------------------
ensemble_spea2_sum_seed2 = minimize(
    ensemble_problem_sum,
    ensemble_algorithm_spea2,
    callback=Spea2Callback(metric='sum', seed=seed2),
    termination=termination_gen,
    seed=seed2,
    verbose=True,
    #verbose=False,
    output=MyOutput(),
    save_history=False,
)
#----------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The optimization took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

n_gen |     lat_min     |     lat_max     |     acc_min     |     acc_max     |     acc_mean   
    1 |  2.37374098E+03 |  2.37374098E+03 |  9.10159988E+01 |  9.10159988E+01 |  9.10159988E+01
    2 |  2.29969812E+03 |  2.36373887E+03 |  9.09940033E+01 |  9.10380020E+01 |  9.10174398E+01
    3 |  2.23167760E+03 |  2.36334953E+03 |  9.09899979E+01 |  9.10459976E+01 |  9.10220197E+01
    4 |  2.18739374E+03 |  2.35203689E+03 |  9.09919968E+01 |  9.10520020E+01 |  9.10262601E+01
    5 |  2.04796180E+03 |  2.34782542E+03 |  9.09619980E+01 |  9.10559998E+01 |  9.10296001E+01
    6 |  2.04366539E+03 |  2.34088772E+03 |  9.09619980E+01 |  9.10660019E+01 |  9.10340199E+01
    7 |  2.00198181E+03 |  2.31573396E+03 |  9.09619980E+01 |  9.10859985E+01 |  9.10358597E+01
    8 |  1.98590643E+03 |  2.29808291E+03 |  9.09560013E+01 |  9.10859985E+01 |  9.10403603E+01
    9 |  1.85819044E+03 |  2.24712453E+03 |  9.09560013E+01 |  9.10960007E+01 |  9.10462606E+01
   10 |  1.80735928E+03 |  2.19009784E+0

In [58]:
del ensemble_spea2_sum_seed2
torch.cuda.empty_cache()

##### seed 3

In [59]:
torch.manual_seed(seed3)
np.random.seed(seed3)
random.seed(seed3)

In [60]:
# start measuring time
start_time = time.time()
#----------------------------
ensemble_spea2_sum_seed3 = minimize(
    ensemble_problem_sum,
    ensemble_algorithm_spea2,
    callback=Spea2Callback(metric='sum', seed=seed3),
    termination=termination_gen,
    seed=seed3,
    verbose=True,
    #verbose=False,
    output=MyOutput(),
    save_history=False,
)
#----------------------------
# stop measuring time
end_time = time.time()
#----------------------------
elapsed = end_time - start_time
print('The optimization took', time.strftime("%Hh%Mm%Ss", time.gmtime(elapsed)), 'to finish.')

n_gen |     lat_min     |     lat_max     |     acc_min     |     acc_max     |     acc_mean   
    1 |  2.37374098E+03 |  2.37374098E+03 |  9.10159988E+01 |  9.10159988E+01 |  9.10159988E+01
    2 |  2.28794335E+03 |  2.36373887E+03 |  9.09899979E+01 |  9.10419998E+01 |  9.10162394E+01
    3 |  2.20760181E+03 |  2.36351911E+03 |  9.09860001E+01 |  9.10480042E+01 |  9.10196199E+01
    4 |  2.14723057E+03 |  2.34877194E+03 |  9.09659958E+01 |  9.10579987E+01 |  9.10219801E+01
    5 |  2.10134532E+03 |  2.34877194E+03 |  9.09659958E+01 |  9.10640030E+01 |  9.10231001E+01
    6 |  2.07844074E+03 |  2.33269496E+03 |  9.09659958E+01 |  9.10820007E+01 |  9.10288001E+01
    7 |  2.00117748E+03 |  2.29695342E+03 |  9.09499969E+01 |  9.10899963E+01 |  9.10302200E+01
    8 |  1.94850826E+03 |  2.28641990E+03 |  9.09280014E+01 |  9.10940018E+01 |  9.10381796E+01
    9 |  1.91530666E+03 |  2.25189236E+03 |  9.09259949E+01 |  9.10960007E+01 |  9.10414400E+01
   10 |  1.75422882E+03 |  2.19732428E+0

In [61]:
del ensemble_spea2_sum_seed3
torch.cuda.empty_cache()

# End of Notebook